<a href="https://colab.research.google.com/github/adityagator/vampire-numbers/blob/master/accuracy84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from skimage.feature import hog
import pickle
import glob
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import *
import random
from sklearn import datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from torch import nn, optim
from sklearn.utils import shuffle
import torch

In [3]:
import torch.nn as nn
class SubClassModel(nn.Module):
    def __init__(self):
        super().__init__()
        # convolution
        # 32 X 32
        self.conv1 = nn.Conv2d(1,3,11)
        self.relu1 = nn.ReLU()     
        self.pool1 = nn.MaxPool2d(2)
        self.batchNorm1 = nn.BatchNorm2d(3) 
        self.dropOut1 = nn.Dropout(p=0.1)
        
        self.conv2 = nn.Conv2d(3,3,26)
        self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(2)
        self.batchNorm2 = nn.BatchNorm2d(3) 

        self.conv3 = nn.Conv2d(3,3,21)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.batchNorm3 = nn.BatchNorm2d(3) 
        self.dropOut3 = nn.Dropout(p=0.1) 
        
        self.conv4 = nn.Conv2d(3,1,10) 
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2)
        self.batchNorm4 = nn.BatchNorm2d(1) 
        
#         self.conv5 = nn.Conv2d(3,3,21) 
#         self.relu5 = nn.ReLU()
# #         self.pool5 = nn.MaxPool2d(2)
#         self.batchNorm5 = nn.BatchNorm2d(3) 
        
        # self.conv6 = nn.Conv2d(3,1,10) 
        # self.relu6 = nn.ReLU()
        # self.pool6 = nn.MaxPool2d(2)
#         self.batchNorm6 = nn.BatchNorm2d(3) 
        
        self.fc1 = nn.Linear(1*8*8, 3, bias=True)
        
        self.outFn = nn.Softmax(dim=1)
        
        
    
    def forward(self,x):
        # print("Before conv: ", x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        # print("Conv1: ", x.shape)
        x = self.pool1(x)
        # print("pool: ", x.shape)
        x = self.batchNorm1(x)
        x = self.dropOut1(x)

        # print("bNorm1: ", x.shape)
        x = self.conv2(x)
        x = self.relu2(x)
        # x = self.pool2(x)
        x = self.batchNorm1(x)
        # print("Conv2: ", x.shape)
       
        x = self.conv3(x)
        x = self.relu3(x) 
        x = self.pool3(x)
        x = self.batchNorm1(x)
        x = self.dropOut3(x)
        # print("Conv3: ", x.shape)
       
        x = self.conv4(x)
        x = self.relu4(x)
        # print("Conv4: ", x.shape) 
        x = self.pool4(x)
        # x = self.batchNorm2(x)
#         x = self.dropOut(x)
        # print("Norm1: ", x.shape)

#         x = self.conv5(x)
#         x = self.relu5(x)
#         x = self.pool5(x)
#         x = self.batchNorm1(x)
        
        # print("Conv5: ", x.shape)
        # x = self.conv6(x)
        # x = self.relu6(x)
        # x = self.pool6(x)
#         x = self.batchNorm1(x)
        # print("After conv:", x.shape)
        
        x = x.view(-1,1*8*8)
        x = self.fc1(x)
        
        x = self.outFn(x)
        return x

In [4]:
Path = "drive/MyDrive/FML_Project/CleanedImages/"
testImages = np.load(Path + "TestingImages.npy") 
testLabels = np.load(Path + "TestingLabels.npy")[0]
trainImages = np.load(Path + "TrainingImages.npy")
trainLabels = np.load(Path + "TrainingLabels.npy")[0]
trainImages, trainLabels = shuffle(trainImages, trainLabels, random_state=0)
print(testImages.shape, trainImages.shape, testLabels.shape, trainLabels.shape)

(600, 200, 200, 3) (5679, 200, 200, 3) (600,) (5679,)


In [5]:
def getGrayScaleImages(images): 
    grayImages =[]
    for image in images: 
        rgbImage = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        grayImage = cv2.cvtColor(rgbImage,cv2.COLOR_RGB2GRAY)
        grayImages.append([grayImage])
    return np.array(grayImages)
grayTrainImages = getGrayScaleImages(trainImages)
grayTestImages = getGrayScaleImages(testImages)
print(grayTrainImages.shape)
print(grayTestImages.shape)

(5679, 1, 200, 200)
(600, 1, 200, 200)


In [6]:
model = SubClassModel() 
lossFn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def performEpoch(input_data, actual_output):
    model.zero_grad()
    # print(input_data.shape)
    prediction_output=model.forward(input_data)
    # print(prediction_output.shape)
    # print(actual_output.shape)
    # print(prediction_output)
    loss = lossFn(prediction_output,actual_output)
    loss.backward()
    optimizer.step()
    return prediction_output, loss

In [7]:
tensoredImages = torch.tensor(grayTrainImages, dtype=torch.float32)
tensoredLabels = torch.tensor(trainLabels-1, dtype=torch.long) 
tensoredtestImages = torch.tensor(grayTestImages, dtype=torch.float32)
tensoredtestLabels = torch.tensor(trainLabels-1, dtype=torch.long) 

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device.type)
if device.type == "cuda":
    tensoredImages = tensoredImages.cuda()
    tensoredLabels = tensoredLabels.cuda()
    tensoredtestImages = tensoredtestImages.cuda()
    tensoredtestLabels = tensoredtestLabels.cuda()
    model = model.cuda()
    lossFn = lossFn.cuda()

cuda


In [9]:
for epoch in range(100): 
    epoch_loss= 0 
    for i in range(20, 5681, 20):
        predicted_output, loss = performEpoch(tensoredImages[i-20:min(i,5679)], tensoredLabels[i-20:min(i,5679)]) 
        epoch_loss += loss 
#     logFile.write("Epoch: "+ str(epoch) + "; Epoch loss: " + str(epoch_loss) + "\n")
    print("Epoch: "+ str(epoch) + "; Epoch loss: " + str(epoch_loss))
# logFile.close()
torch.save(model, "TrainedModelCNN")
torch.save(model, Path + "TrainedModelCNNOptimistic")

Epoch: 0; Epoch loss: tensor(307.8743, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 1; Epoch loss: tensor(308.1456, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2; Epoch loss: tensor(307.1827, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 3; Epoch loss: tensor(299.7833, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 4; Epoch loss: tensor(294.9713, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 5; Epoch loss: tensor(290.8351, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 6; Epoch loss: tensor(287.2568, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 7; Epoch loss: tensor(272.0703, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 8; Epoch loss: tensor(264.4037, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 9; Epoch loss: tensor(257.5373, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 10; Epoch loss: tensor(252.6871, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 11; Epoch loss: tensor(248.1357, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 12; Epoch loss: tensor(246.1223

In [10]:
predicted_output, loss = performEpoch(tensoredImages, tensoredLabels) 
print(loss)

predictedLabels= np.array([np.argmax(prediction) for prediction in predicted_output.cpu().detach().numpy()]) 
cf = confusion_matrix(predictedLabels, trainLabels-1) 
print(cf)

tensor(0.6609, device='cuda:0', grad_fn=<NllLossBackward>)
[[1142  167   63]
 [  64 1836   35]
 [ 199  105 2068]]


In [11]:
prediction = model(tensoredtestImages)
print(prediction.shape)

torch.Size([600, 3])


In [12]:
from sklearn.metrics import accuracy_score
predictedTestLabels= np.array([np.argmax(pred) for pred in prediction.cpu().detach().numpy()])  
cf = confusion_matrix(predictedTestLabels, testLabels-1) 
print(cf)
accuracy = accuracy_score(testLabels-1, predictedTestLabels)
print(accuracy)

[[131  24  11]
 [ 20 164   3]
 [ 49  12 186]]
0.8016666666666666


In [ ]:
for epoch in range(100): 
    epoch_loss= 0 
    for i in range(20, 5681, 20):
        predicted_output, loss = performEpoch(tensoredImages[i-20:min(i,5679)], tensoredLabels[i-20:min(i,5679)]) 
        epoch_loss += loss 
torch.save(model, Path + "TrainedModelCNNOptimistic")